# Q1 Which is a better Estimator

MCMC is one of estimator under Monte Carlo Simulations but it is not certainly the best.<br>
As we move towards a more stationary Distribution, we take more number of iterations to reach convergence.<br>
var($X_t(from MCMC)$) = Var(X) + 2 $\sum_{t=1}^{\infty}cov(X_0,X_t)$<br>
This will be abviously more than that of normal iid sample.<br>
Hence, it is not a better sampler.


# Q2. 

f(y|$\mu$) = N($\mu$,1)<br>
f($\mu$|$\nu$) = $t_{\nu}$<br>
f($\nu$) = Truncated Gamma $(a_0, b_0, (2, \infty ) )$<br><br>
Joint Distribution $(\mu, \nu):f(\mu, \nu | y)$<br>
f($\mu, \nu$ | y) $\propto$ f(y|$\mu$) f($\mu$|$\nu$) f($\nu$)<br>
f($\mu, \nu$ | y) $\propto$ $\nu^{a_0-1}e^{(\frac{-\sum_i(y_i-\mu)^2}{2}-\frac{\nu}{b_0})}(1+\frac{\mu^2}{\nu})^{-\frac{\nu+1}{2}}I(2 < \nu < \infty)$

## MH Algorithm:<br>
1. Set initial value of $\mu, \nu$ from the above data.
2. Draw a random sample from gaussian proposal for k and random proposal for l
3. Calculate MH Ratio $\alpha$
4. Compare the ratio with Uniformly sampled value between 0 and 1. If U < $\alpha$, then update the value with k, l
5. Else update the sample values.

In [1]:
using Random
using QuadGK
using Distributions

In [2]:

a = 2
b = 0.1
n = 100
v = rand(Truncated(Gamma(a,b),2,Inf))
u = rand(TDist(v))
Y = rand(Normal(u,1),n)
println("Prior u which is unknown = u = ",u)
println("Prior v which is unknown = v = ",v)

Prior u which is unknown = u = -0.020763903500327495
Prior v which is unknown = v = 2.0173011873112867


In [3]:
f(r,t) = exp(-0.5*sum((Y.-r).^2))*((1+(r*r)/t)^(-0.5*(t+1)))*((t)^(a-1))*exp(-1.0*t*b);
u = mean(Y);
v = a/b;
iterations = 100000;
for t=0:iterations
    k = rand(Normal(u,1))
    l = rand(Uniform(2,v+0.01))
    alpha = min(1,(1.0*f(k,l))/f(u,v))
    U = rand()
    if U<alpha
        u = k
        v = l
    end
end
println("Estimated u = ",u)
println("Estimated v = ",v)




Estimated u = -0.07304953322385899
Estimated v = 2.028811452083325


# Q3 

f(y|$\beta, \sigma ^2$) = N($X\beta$,$\sigma ^2$)<br>
f($\beta$|$\sigma ^2$) = $N_{p}(\mu, \sigma ^2)$<br>
f($\sigma ^2$) = Inverse Gamma $(a,b)$<br><br>
Joint Distribution $(\mu, \nu):f(\mu, \nu | y)$<br>
f($\beta, \sigma ^2$ | y) $\propto$ f(y|$\beta, \sigma ^2$) f($\beta$|$\sigma ^2$) f($\sigma ^2$)<br>

In [74]:
using CSV
using Tables
using LinearAlgebra
data = CSV.File("boston.csv") |> Tables.matrix;

In [75]:
n = size(data,1)
p = size(data,2)
a = 2
b = 1

1

In [83]:
sigma = rand(InverseGamma(a,b))
mu = rand(p)
beta = rand(MvNormal(mu,sigma*Matrix(I,p,p)))
Y = rand(MvNormal((data*beta),sigma*Matrix(I,n,n)))
Invp(s) = inv(s*Matrix(I,p,p));
Invn(s) = inv(s*Matrix(I,n,n));
f(BETA, SIGMA) = exp.(-(transpose(Y.-data*BETA)*Invn(SIGMA)*(Y.-data*BETA)))*exp.(-(transpose(BETA-mu)*Invp(SIGMA)*(BETA-mu)))*((1/SIGMA)^(-(a+1)))*exp(-b/SIGMA)

f (generic function with 1 method)

In [84]:
BETA = pinv(data)*Y
# println(BETA)
SIGMA = b/(a-1)
iterations = 1000
for i=0:iterations
    β = rand(MvNormal(BETA,SIGMA*Matrix(I,p,p)))
    x = rand(Normal(SIGMA,1))
    alpha = min(1,f(β,x)/f(BETA,SIGMA))
    U = rand()
    if U<alpha
        count = count+1
        BETA = β
        SIGMA = x
    end
end
println(BETA)
println(SIGMA)

[-0.08522359041002138, 0.5151035089355178, 1.0248381393826553, 1.332759546087261, 1.7611513547130007, 1.551999650624613, -0.009703016378105504, -0.48295291820752206, -0.5464342609152866, -0.1491000280865715, 1.0221855645257956, 0.3618695066828239, -0.017451220264118942, 0.2118208370379633]
1.0
